In [1]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor

### Helper Functions

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['Date'] = pd.to_datetime(data.Date)
    data['DOLocationID'] = data['DOLocationID'].astype(str)
    print('Days: ',len(set(data.Date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='vehicle_count', index=['Date','Hour'],
                    columns=['DOLocationID'], aggfunc=np.sum, fill_value=0)
    return table

In [4]:
def zscoreNormalizeSpatial(matrix):
    m = matrix.copy()
    for i in range(m.shape[0]):
        m[i, :] = (m[i, :] - m[i, :].mean()) / (m[i, :].std()+1e-10)
        
    return m

In [5]:
def standardize(matrix):
    m = matrix.copy()
    scaler = StandardScaler()
    scaler.fit(m)
    t = scaler.transform(m)
    return scaler, t

In [6]:
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)

In [7]:
def addLag(dataset, maxlag, lagColumns):
    dataset_list = [dataset]

    for l in range(1, maxlag+1):
        df = dataset.shift(l)
        df = df[lagColumns]
        df.columns = [c+'_lag_'+str(l) for c in df.columns]
        dataset_list.append(df)

    dataset = pd.concat(dataset_list, axis=1).dropna()
    return dataset

In [8]:
def get_rmse(matrix1, matrix2):
    sumSquareError = np.mean(np.power(matrix1 - matrix2,2))
    rmse = np.power(sumSquareError,0.5)
    return rmse

In [9]:
def get_weights(rawdata, zontoBorough):
    
    rawdata['Borough'] = rawdata['DOLocationID'].apply(lambda x:zontoBorough[x])
    
    borough_df = rawdata[['vehicle_count','Borough']].groupby(by='Borough').sum().reset_index()

    zone_df = rawdata[['vehicle_count','DOLocationID']].groupby(by='DOLocationID').sum().reset_index()

    zone_df['Borough'] = zone_df['DOLocationID'].apply(lambda x:zontoBorough[x])

    zone_df = pd.merge(borough_df, zone_df, on=['Borough'], how='inner')

    zone_df['zone_weight'] = zone_df.vehicle_count_y / zone_df.vehicle_count_x

    zone_df = zone_df[['Borough', 'DOLocationID', 'zone_weight']]

    return zone_df

#### Load Raw Data

In [10]:
hub = 'Penn'
tune_hyp_params = True

In [11]:
dataDir = '/home/urwa/Documents/Projects/NYU Remote/project/data/processedData/'
file = dataDir + hub + 'VehiceByHour.csv'

In [12]:
rawdata = loadData(file)

Raw shape:  (2251320, 4)
Days:  365


In [13]:
rawdata.head(2)

,DOLocationID,Date,Hour,vehicle_count
0,1,2018-01-01,0,0.0
1,2,2018-01-01,0,0.0


In [14]:
edge_data = getTimeSeries(rawdata)
edge_data = edge_data.reset_index()
edge_data.head(3)

DOLocationID,Date,Hour,1,10,100,101,102,106,107,108,...,90,91,92,93,94,95,96,97,98,99
0,2018-01-01,0,0,0,8,0,0,2,14,0,...,11,1,0,0,0,1,0,0,0,0
1,2018-01-01,1,0,0,8,0,0,0,70,0,...,11,0,0,1,0,3,0,2,0,0
2,2018-01-01,2,1,0,10,0,0,2,57,0,...,20,0,0,0,0,1,0,1,1,0


### Merge External Data Features

In [15]:
externalDataDir = "/home/urwa/Documents/Projects/NYU Remote/project/data/HongData/"
extFile = externalDataDir + hub.upper() + ".csv"

In [16]:
extDf = pd.read_csv(extFile)
print(extDf.shape)
extDf.head(2)

(8760, 46)


,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,18/1/1 0:00,0,147,319,466,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
1,18/1/1 1:00,1,347,397,744,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0


In [17]:
extDf['date'] = pd.to_datetime(extDf['date'], yearfirst=True)
extDf.head(2)

,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01 00:00:00,0,147,319,466,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
1,2018-01-01 01:00:00,1,347,397,744,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0


In [18]:
min(extDf.date), max(extDf.date)

(Timestamp('2018-01-01 00:00:00'), Timestamp('2018-12-31 23:00:00'))

In [19]:
extDf['Hour'] = extDf['date'].dt.hour
extDf['Dow'] = extDf['date'].dt.dayofweek
extDf['Date'] = extDf['date'].dt.date

In [20]:
extDf.columns

Index(['date', 'arrival', 'fhv', 'yellow', 'vehicle', 'ifmon', 'iftue',
       'ifwed', 'ifthu', 'iffri', 'ifsat', 'ifsun', 'if0', 'if1', 'if2', 'if3',
       'if4', 'if5', 'if6', 'if7', 'if8', 'if9', 'if10', 'if11', 'if12',
       'if13', 'if14', 'if15', 'if16', 'if17', 'if18', 'if19', 'if20', 'if21',
       'if22', 'if23', 'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow', 'Hour', 'Dow',
       'Date'],
      dtype='object')

In [21]:
selected_columns = ['Date', 'Hour', 'Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

In [22]:
extDf = extDf[selected_columns]

In [23]:
print(edge_data.shape)
print(extDf.shape)

(8760, 259)
(8760, 14)


In [24]:
edge_data['Date'] = pd.to_datetime(edge_data['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])

In [25]:
edge_data = pd.merge(edge_data,extDf, on=['Date', 'Hour'], how='inner')
print(edge_data.shape)
edge_data['Date'] = edge_data['Date'].dt.date
edge_data.head()

(8760, 271)


,Date,Hour,1,10,100,101,102,106,107,108,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01,0,0,0,8,0,0,2,14,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
1,2018-01-01,1,0,0,8,0,0,0,70,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
2,2018-01-01,2,1,0,10,0,0,2,57,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
3,2018-01-01,3,5,0,3,0,0,0,26,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
4,2018-01-01,4,3,0,7,0,0,0,8,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0


In [26]:
edge_data.columns

Index(['Date', 'Hour', '1', '10', '100', '101', '102', '106', '107', '108',
       ...
       'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd', 'cdd',
       'participation', 'newsnow', 'snowdepth', 'ifSnow'],
      dtype='object', length=271)

In [27]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in edge_data.columns if c not in ext_columns and \
                c not in DateColumns and c != 'Hour']

lagColumns = targetColumns + ['arrival']

In [28]:
maxlag = 12

edge_data_lag = addLag(edge_data, maxlag, lagColumns)

edge_data_lag.shape

(8748, 3367)

In [29]:
CommR2List = []
EdgeR2List = []
residualDf_list = []

for m in range(1,13):
    print()
    print("month: ",m)
    month_index  = pd.to_datetime(edge_data_lag.Date).dt.month == m

    dataset_train = edge_data_lag[~month_index]
    dataset_test = edge_data_lag[month_index]
    print("Train Size: ",dataset_train.shape)
    print("Test Size: ",dataset_test.shape)

    X_train = dataset_train.drop(targetColumns+DateColumns , axis = 1)
    X_test = dataset_test.drop(targetColumns+DateColumns , axis = 1)
    y_train = dataset_train[targetColumns]
    y_test = dataset_test[targetColumns]

    rf2 = RandomForestRegressor(random_state = 2019, n_estimators=150, 
                               min_samples_split=3,
                               min_samples_leaf= 2, 
                               max_features= 'sqrt',
                               max_depth= None, 
                               bootstrap= False)

    rf2.fit(X_train,y_train)

    print("Train R2: ",rf2.score(X_train,y_train))
    test_r2 = rf2.score(X_test,y_test)
    print("Test R2: ",test_r2)


    prediction = rf2.predict(X_test)


    residual = y_test - prediction
    residual_df = dataset_test[['Date','Hour']]
    residual_df = pd.concat([residual_df,pd.DataFrame(residual)], axis =1)


    edge_r2 = r2_score(y_test, prediction, multioutput='variance_weighted')
    print("Edge R2: ",edge_r2)

    CommR2List.append(test_r2)
    EdgeR2List.append(edge_r2)
    residualDf_list.append(residual_df)


month:  1
Train Size:  (8016, 3367)
Test Size:  (732, 3367)
Train R2:  0.9569625762958458
Test R2:  0.7072323079926246
Edge R2:  0.7072323079926246

month:  2
Train Size:  (8076, 3367)
Test Size:  (672, 3367)
Train R2:  0.9565351076382644
Test R2:  0.7352199613830063
Edge R2:  0.7352199613830063

month:  3
Train Size:  (8004, 3367)
Test Size:  (744, 3367)
Train R2:  0.9570275239054513
Test R2:  0.7129631865264942
Edge R2:  0.7129631865264942

month:  4
Train Size:  (8028, 3367)
Test Size:  (720, 3367)
Train R2:  0.9564944184077502
Test R2:  0.770081556280708
Edge R2:  0.770081556280708

month:  5
Train Size:  (8004, 3367)
Test Size:  (744, 3367)
Train R2:  0.9568656459809336
Test R2:  0.7731036994298971
Edge R2:  0.7731036994298971

month:  6
Train Size:  (8028, 3367)
Test Size:  (720, 3367)
Train R2:  0.9568927344264454
Test R2:  0.7507262851925645
Edge R2:  0.7507262851925645

month:  7
Train Size:  (8004, 3367)
Test Size:  (744, 3367)
Train R2:  0.9568714149575245
Test R2:  0.74557

In [30]:
print(np.mean(CommR2List))
print(np.mean(EdgeR2List))

0.7377892564299263
0.7377892564299263


In [31]:
res_df = pd.concat(residualDf_list, axis = 0)
print(res_df.shape)
res_df.head()

(8748, 259)


,Date,Hour,1,10,100,101,102,106,107,108,...,90,91,92,93,94,95,96,97,98,99
12,2018-01-01,12,5.951111,-0.104444,2.326667,-0.015556,-0.051111,0.810000,-4.761111,-0.028889,...,-2.244444,-0.110000,0.662222,2.718889,-0.052222,-0.186667,-0.012222,-0.605556,-0.026667,0.000000
13,2018-01-01,13,2.092222,-0.097778,1.827778,-0.040000,0.940000,-0.241111,5.887778,-0.030000,...,0.257778,-0.120000,0.520000,-0.282222,-0.075556,0.716667,-0.023333,1.191111,-0.036667,0.000000
14,2018-01-01,14,3.902222,-0.140000,-9.011111,-0.017778,-0.044444,-0.315556,5.332222,-0.021111,...,-1.734444,-0.117778,-0.416667,0.696667,-0.060000,-0.321111,-0.018889,-0.953333,-0.040000,0.000000
15,2018-01-01,15,3.545556,-0.171111,2.031111,-0.036667,-0.081111,1.661111,10.402222,-0.023333,...,-4.933333,-0.107778,-0.526667,-0.217778,-0.056667,0.631111,-0.002222,2.020000,-0.036667,-0.002222
16,2018-01-01,16,4.665556,-0.210000,5.880000,-0.020000,-0.102222,0.627778,6.074444,-0.048889,...,-2.193333,0.858889,-0.507778,-0.172222,-0.064444,0.601111,-0.007778,2.928889,-0.047778,-0.002222


In [32]:
dateCols = ['Date', 'Hour']
otherCols = [c for c in res_df.columns if c not in dateCols]

res_df['residual'] = np.sum(res_df[otherCols].values, axis=1)

res_df = res_df[dateCols+['residual']]
print(res_df.shape)

(8748, 3)


In [33]:
res_df.to_csv('/home/urwa/Documents/Projects/NYU Remote/project/data/residuals/penn_edge.csv')